# ALeRCE SN checker and reporter tutorial

Francisco Förster on behalf of the ALeRCE collaboration

Nov 6th, 2019

In [1]:
# load some modules to change the notebook format and to be able to load videos later
from IPython.display import HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Introduction

ALeRCE is the Automatic Learning for the Rapid Classification of events, see http://alerce.science/.

ALeRCE is an astronomical broker built to ingest, aggregate, annotate and classify astronomical alerts.

Astronomical alerts are packets produced by a new generation of large etendue telescopes reporting any significant change in the sky, e.g. transients, variable stars, asteroids.

ALeRCE has two types of classifiers: an early classifier which uses only one image to classify objects, and a late classifier which uses the flux time series or light curve to classify objects.

The early classifier classifies objects in the following classes:

* supernova (extragalactic transients)
* variable stars
* asteroids
* active galactic nuclei
* bogus

The objective of this notebook is to teach you how to check supernova candidates in the early classifier using the [SN hunter](http://dev.alerce.online/) tool, and how report them to the Transient Name Server ([TNS](https://wis-tns.weizmann.ac.il/search)) for the community to follow them up.

### Before you start

You will need to do the following:

* install the alerce api (`pip install alerce`)
* clone the [TNS_upload](https://github.com/alercebroker/TNS_upload) repository (e.g. `git clone git@github.com:alercebroker/TNS_upload.git`). If you have already cloned it remember to do a `git pull`, which will also give you the latest version of this tutorial.
* ask [me](francisco.forster@gmail.com) for the TNS API key (please use with caution!!)

# Contents

This tutorial is divided in the following steps:

1. How to check candidates in the SN hunter
2. How to gather and filter candidates selected by other users
3. How to select their host galaxies
4. How to report these candidates with their host galaxies to TNS

# 1. Check candidates in the SN hunter

The SN hunter is a tool to visualize the best SN candidates reported by the ALeRCE early classifier. It has two versions: the [production](https://snhunter.alerce.online/) and the [development](http://dev.snhunter.alerce.online/) websites. 

Both [production](https://snhunter.alerce.online/) and [development](http://dev.snhunter.alerce.online/) allow you to see recent SN candidates produced by ZTF. 

The [development](http://dev.snhunter.alerce.online/) version also allows you to report bogus candidates (Nov 2019). In the future you will also be able to report SN candidates.

When you enter the snhunter you will see something like this. 

In the development version you can login using your google account to be able to report bogus candidates (upper right corner)

![](images/snhunter.png)

You can do the quick start tutorial to get some quick description of the different components.

### Top 100 SN candidates table

The table on the right shows the following information of the top 100 SN candidates:

* the object IDs (the unique identifier for each object)
* the discovery date
* a score which indicates the probability of the candidate of being SN
* the number of detections

You can sort by any of these quantities by clicking in the column title. 

You can also select how old the candidates can be by selecting an the maximum age of the candidates in the selection bar on the left of the refresh button.

The refresh button allows you to update the table taking into account the most recent candidates (consider that the processing from the previous night usually stops around noon in Chile)

![](images/snhunter_table.png)

### Celestial map



The map on the left shows the location of the candidates in the sky. 

The size of the circles indicate the score. 

The Milky Way is shown as a gray patch and the ecliptic is shown as a green line. One shouldn't expect to see SN candidates in the galactic plane, although there are exceptions. Asteroids can be confused with SN around the ecliptic. 

If you select one candidate from the table the map will be centered around it, which is shown as a blue circle. The size of the circle indicates the score of the candidate.

![](images/snhunter_map.png)

### Candidate information

When you click in a candidate from the table you will also see more information about the candidate on top of the map and table.

On the left panel you will see some statistics: 

   - the object ID, the coordinates
   - the detection band, the magnitude and time of detection
   - PanSTARRS crossmatch information: ID, distance and a star galaxy score of the nearest source in PanSTARRS
   - a link to the ALeRCE explorer for the same object id
   - a query to the NASA Extragalactic Explorer (NED) for the same position
   - a query to the Transient Name Server (TNS) around the position of the candidate
   - a query to Simbad Astronomical Database around the position of the candidate
   - a link to see the full alert information from ZTF

In the middle panel you will see archival images around the position of the object from Aladin (by default showing color images from PanSTARRS)

In the right panel you will see the science, template and difference stamps centered around the candidate. Considering that the candidate will appear in the science image and that the difference requires a convolution step which tends to smooth features, it is recommended to look first at the science image, then the template, then the difference image. 


![](images/snhunter_candidate.png)

In the previous example you can see that the nearest object in PanSTARRS is at ~7 arcsec and is probably a galaxy because its score is 0.004. This is consistent with what is seen in the Aladin and ZTF images of the host and the candidate (ZTF pixels are about 1").

### Full alert information

If you click on the Full alert information you will see the following window:

![](images/snhunter_full.png)

Where you can scroll to see different properties of the alert. The explanation for the different fields is contained in the schema description on the top.

Some important fields are:

1. `isdiffpos`: whether the difference is positive (t) or not (f). If the difference is not positive it shouldn't be reported as SN.
2. `drb`: deep learning based real (1) bogus (0) score
3. `rb`: random forest based real (1) bogus (0) score
4. `candid`: unique identifier of the alert
5. `fwhm`:source full width half maximum

# Some selection criteria

### Good SN candidates:

* should be **located on top or near a galaxy**. Sometimes it is better to see the Aladin image to look for a nearby galaxy. Bear in mind that galaxies can be more extended than what the images suggest. If you don't see a host galaxy do not report it, it is most likely a variable star (if it coincides with the position of a star) or an asteroid (specially if it is near the ecliptic).

* should be **visible in the science image**. If you cannot see the candidate in the science image or if you are not sure you should always look at the `isdiffpos` field in the full alert information to make sure the alert has a positive difference. If the difference looks fine and `isdiffpos` is True then you can report it even if you cannot clearly distinguish it in the science image, e.g. if the candidate is in the center of a galaxy.

* should have a **shape similar to other stars, with fuzzy edges and a symmetric shape** in the field in the science image. If you cannot see other stars and you are not sure about a candidate it is useful to look at the `drb` field (it should be close to one), although this doesn't always work. If the candidate has very sharp edges in the science image it is most likely a cosmic ray. This is specially the case if it also has a non symmetric shape. If you are unsure do not report.

* if the candidate is close to a galaxy, but it doesn't look convincing, check if it has **more than one detection**. In that case you can click in the ALeRCE explorer link and look at the images from the second detections there.

### Bad candidates:

* if the candidate **coincides with the position of a star** do not report it, it is most likely a variable star.
* if the candidate is in a **very crowded field** do not report it, unless you can clearly separate the host galaxy and the candidate from the rest
* if the **science or template images look wrong around the position of the candidate** (bad pixel, bad column, cosmic ray, ghosts, bright star spikes, see more detail below) do not report it.
* if you see **many candidates with the same detection time**, where you cannot separate the candidate in the science image it is most likely due to a calibration problem. For this reason it is recommended to **sort by detection date** when checking candidates.

### Bogus candidates:

If you are using the developer's version of the snhunter you can report bogus candidates. This helps building training sets to improve the classifier.

To understand what to report and what not to report consider that the early classifier tries to classify point like sources with an astrophysical origin into the classes supernova (extragalactic transients), active galactic nuclei, variable stars and asteroids. The bogus class corresponds to candidates without an astrophysical origin or things which do not look point source like.

Possible origins for bogus objects are:

1. **dipoles**: when images are not well aligned the subtraction looks like a dipole with negative and positive values

![](images/snhunter_dipole01.png)
![](images/snhunter_dipole02.png)

2. **cosmic rays**: energetic particles which hit the detector directly without going through the optics of the telescope. They are characterized for having sharp edges and non-symmetric shapes. Note that they can be distinguished more easily in the science image.

![](images/snhunter_cr01.png)
![](images/snhunter_cr02.png)
![](images/snhunter_cr03.png)
![](images/snhunter_cr04.png)

3. **bright star spikes**: when a bright star is present one can see radial spikes originating from the bright star, which are produced by the element holding the secondary mirror (spider)

![](images/snhunter_spike.png)

4. **bad columns**: when a whole column or row has problems in the detector

![](images/snhunter_badcolumn.png)

5. **hot pixels**: when one or a group of pixels have more counts than normal. They look like cosmic rays.

![](images/snhunter_hotpixel.png)

5. **cold pixels**: when one or a group of pixels have less counts than normal

![](images/snhunter_coldpixel.png)

## Examples of good candidates

![](images/snhunter_sn01.png)
![](images/snhunter_sn02.png)
![](images/snhunter_sn03.png)
![](images/snhunter_sn04.png)
![](images/snhunter_sn05.png)

# Examples of bad candidates which turned out to be real

Some candidates look bad, but subsequent detections show that they were real. My recommendation is not to report them, but this should be evaluated case by case.

This object has 4 detections, the first detection is asymmetric and with sharp edges. I would have been doubtful and probably not reported it.
![](images/snhunter_uglysn01.png)

This object has 10 detections, the first detection is really ugly. I would not have reported this at all!
![](images/snhunter_uglysn02.png)

### Example video

Here you can see a [video](https://www.youtube.com/embed/WG24B5xgNwo) of the candidate selection process

In [2]:
HTML('<iframe width="1200" height="800" src="https://www.youtube.com/embed/WG24B5xgNwo" frameborder="0" allowfullscreen></iframe>')

# 2. Gather and filter candidates selected by other users

When checking candidates:

* wait until ~noon Chile time for the ZTF night to end (you can use our [grafana](https://grafana.alerce.online/?orgId=1) tool to check whether candidates are still been received)
* select the last 24 hr of candidates
* sort them by detection time, this will help you identify clusters of bad candidates in time, but also report in chronological order, which will facilitate the comparison between list of candidates reported by different people


Reporting in slack

* When exploring candidates in the SN hunter we recommend to open the ALeRCE explorer for those candidates which looked interesting. This will allow you to look at the non-detections and possible other detections, or to look for crossmatches.

* While you do this you should go to the #recentsupernova channel in the [alerce slack](https://alerce-science.slack.com) and start a thread with the current date in the format `yyyymmdd (your initials)`, e.g. `20191105 (FF)`

* In this thread you should start copying and pasting the links to the individual candidates that you selected. Other people can then comment on your candidates, e.g.:

![](images/snhunter_slack_FEB.png)

or 

![](images/snhunter_slack.png)



See video below:

#### Example [video](https://www.youtube.com/embed/rLAwCRams6U)

In [3]:
HTML('<iframe width="1100" height="700" src="https://www.youtube.com/embed/rLAwCRams6U" frameborder="0" allowfullscreen></iframe>')

Final list of candidates:

* there will be a group of people checking candidates every day, you should make sure you wait for their list and comment their candidates. 

* one person will decide which candidates to accept or not. This person will be responsible for submitting them to the Transient Name Server

* when the final list of candidates is compiled this person will report them using the format `yyyymmdd (final)`, e.g.

![](images/snhunter_slack_final.png)

Note that the candidates ZTF19acoqzib and ZTF19acorgwx from FEB were removed, as well as the candidate ZTF19acoqacu from FF.


#### Example [video](https://www.youtube.com/embed/_K9RKh92HOY)

In [4]:
HTML('<iframe width="1100" height="700" src="https://www.youtube.com/embed/_K9RKh92HOY" frameborder="0">')

# 3. Select host galaxies

Once the final list of candidates has been compiled you should save those candidates in the `TNS_upload` repository, in the directory `candidates` and in a file named `yyyymmdd.txt` containing just the object ids, e.g.

```
TNS_upload/candidates$ cat 20191105.txt 
ZTF19acopvmc
ZTF19acopyaq
ZTF19acoqxag
ZTF19acotweu
ZTF19acouclc
ZTF19acouhpm
```

If you find out later that day that there were additional candidates which you would like to submit you can create a file a second file, e.g. `20191105b.txt`

This list will be used in the [TNS upload notebook](https://github.com/alercebroker/TNS_upload/blob/master/ALeRCE_ZTF_TNS_send_SNe.ipynb) to select the candidate host galaxies and send a report to Transient Name Server (TNS)

### Using the [TNS upload notebook](https://github.com/alercebroker/TNS_upload/blob/master/ALeRCE_ZTF_TNS_send_SNe.ipynb)

Assuming that you have cloned the notebook and assuming that you have the previously described file in the `candidates` directory, the the candidate selection happens in the following cell (you must modify the `refstring` string to make sure you are reading the right file):

![](images/snhunter_TNS_list.png)



In order to select host galaxies you need to execute the following cell:

![](images/snhunter_hosts.png)


This will allow you to interact with the NED, Simbad and SDSSDR15 catalogs (see video bellow)

The candidate selection works as follows:

1. For every candidate an Aladin window will be opened, centered around the position of the candidate.
2. You will see queries to NED, Simbad and SDSSDR15 being done (or catsHTM and Vizier if these options are set to True)
3. When these queries are perfomed you will see a catalog of objects overlaid on top of the Aladin image. The candidate position with its object ID will also be shown.
4. You should identify the likely host galaxy and hover around its associated catalog object
4. If you hover your mouse over different catalog objects you will see information like the object ID, the source catalog, whether the object is a galaxy, and whether it has a redshift.
5. If the object was in the SDSSDR15 catalog it will query SDSSDR15 to obtain a photometric redshift (this takes a few seconds per object). This query will be done just once, it will not be repeated when hovering again or running the cell again.
6. Any appearances of the word *Redshift* (NED) or *z_value* (Simbad) will be shown in red and larger fonts to help you identify spectroscopic redshifts.
7. Any appearances of the word *photoz* (SDSSDR15) will be shown in orange and larger fonts to help you identify photometric redshifts.
8. When you have identified the likely host galaxy and its associated catalog object carefully click **once** over it. It is OK to zoom and pan over some region to be able to see the different catalog objects.
10. If the same galaxy appears in different catalogs give preference to objects which have redshift, then photoz, then some ID. If there is no catalog object just click in the SN candidate position.

#### Example [video](https://www.youtube.com/embed/oGt9dUSIsqE)

In [5]:
HTML('<iframe width="1100" height="700" src="https://www.youtube.com/embed/oGt9dUSIsqE" frameborder="0">')

Note that in minute 1:25 the Simbad query did not work. When this happens you need to wait until the query stops with a timeout.

If you see some errors in your table of host galaxies you can apply manual modifications in the "Manual modifications" cell

# 4. Report to TNS

After you select the host galaxies you need to prepare the report. 

You will need to prepare the json file and then submit to TNS.

#### TNS API

Here you will need the TNS API key used by ALeRCE. This allows you to query the TNS database directly and, more importantly, to submit candidates!

Please write me (francisco.forster@gmail.com) if you want to use the key for the sandbox (test evironment) or official site (what everyone will see). Note that we have had problems with the sandbox environment before.

You may want to read the [TNS bulk report manual](https://wis-tns.weizmann.ac.il/sites/default/files/api/TNS_bulk_reports_manual.pdf)

You can select the test environment with the `sandbox = True` variable.

**If you select `sandbox=False` you will be sending candidates to the official site!**

![](images/snhunter_api.png)

#### Author list and json file

To prepare the json file you need to first make sure the list of authors is correct. Normally we would include the following people:

1. Person responsible for sending to TNS, e.g.

   F. Förster


2. People helping select candidates, e.g.

    F.E. Bauer, G. Pignata, 


3. People involved in developing the tools required to select candidates, e.g.

    J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Est\'evez, Huijse, P, E. Reyes, I. Reyes, P. S\'anchez-S\'aez, C. Valenzuela,
    E. Castillo, D. Ruz-Mieres, D.V. Rodríguez
    
    
4. People leading groups in ALeRCE:

    M. Catelán, S. Eyheramendy, M.J. Graham

#### Report

Then you can prepare the report where we will query TNS to check if candidates are known. Here we will also compute the rise times. 

![](images/snhunter_TNS_report.png)

As you loop over all the candidates the following information will be gathered for every candidate:

* the ALeRCE groupid, instrument, units, type of report, coordinates, discovery date
* the host galaxy properties
* the available detections
* the available non-detections
* the magnitude rise rate in each band (only reported if significantly larger than 0.05 mag/day)
* whether the object is known in TNS. If known, it will not be reported (in this case `counter` variable does not increase, see image below)
* the candidate stamps are shown

This is your last chance to remove candidates, which you can do manually modifying the for loop above.


![](images/snhunter_tns_candidates.png)

You should paste in slack everything rising faster than 0.25 mag/day, these are potentially very young candidates.

![](images/snhunter_fastrisers.png)

#### Sending to TNS

After the previous steps you can send the candidates to TNS.

You should query for the status of the submission and you should see an OK message.

After this check in the TNS website: https://wis-tns.weizmann.ac.il/search

![](images/snhunter_tns_send.png)

You may need to wait a few seconds before your report appears in the TNS website.

If something goes wrong with TNS, e.g. you get an OK message, but no results are shown in the website (in the sandbox this is common, but in the main site this has happened only once), you should contact 
![](images/snhunter_tns_contact.png)


#### Example [video](https://www.youtube.com/embed/mNHY98bEN14)

In [6]:
HTML('<iframe width="1100" height="700" src="https://www.youtube.com/embed/mNHY98bEN14" frameborder="0">')

## Congratulations! You have managed to submit ALeRCE candidates to TNS!

As a last note, if you find a very nice candidate perhaps you can contact your astronomy friends to get a confirmation image and an early spectrum!